# KEGG enrichment of stable genes

This notebooks looks at the group of most and least stable genes and performs a KEGG enrichment analysis to determine if there are any KEGG pathways that are significantly over-represented in our most or least stable gene sets.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import pandas as pd
import numpy as np
import scipy.stats
import statsmodels.stats.multitest
from scripts import paths, utils, annotations

In [2]:
# Load KEGG pathway data
pao1_pathway_filename = "https://raw.githubusercontent.com/greenelab/adage/7a4eda39d360b224268921dc1f2c14b32788ab16/Node_interpretation/pseudomonas_KEGG_terms.txt"

In [3]:
pao1_pathways = annotations.load_format_KEGG(pao1_pathway_filename)
print(pao1_pathways.shape)
pao1_pathways.head()

(169, 2)


,1,2
0,,
KEGG-Pathway-pae00072: Synthesis and degradation of ketone bodies,10,"{PA3925, PA2003, PA2553, PA2001, PA3589, PA478..."
KEGG-Pathway-pae00071: Fatty acid degradation,32,"{PA3454, PA0879, PA4994, PA1748, PA2574, PA118..."
KEGG-Pathway-pae00903: Limonene and pinene degradation,9,"{PA3331, PA1748, PA1027, PA3014, PA4899, PA182..."
KEGG-Pathway-pae00380: Tryptophan metabolism,27,"{PA2080, PA1748, PA3589, PA3426, PA1736, PA208..."
KEGG-Pathway-pae00900: Terpenoid backbone biosynthesis,16,"{PA4557, PA3650, PA4044, PA4669, PA3652, PA392..."


In [4]:
# Load transcriptional similarity df
# These are the subset of genes that we will consider
pao1_similarity_scores_filename = "pao1_similarity_scores.tsv"

pao1_similarity_scores = pd.read_csv(
    pao1_similarity_scores_filename, sep="\t", header=0, index_col=0
)

In [5]:
pao1_similarity_scores.head()

,PA14 homolog id,Transcriptional similarity across strains,P-value,Name,label
PAO1 id,,,,,
PA1950,PA14_39280,0.353718,1.962739e-157,rbsK,NaN
PA4872,PA14_64440,0.381711,4.085891e-185,NaN,NaN
PA3680,PA14_16770,0.381384,8.943350e-185,NaN,NaN
PA0586,PA14_07650,0.542184,0.000000e+00,NaN,most stable
PA1468,PA14_45450,0.365443,1.080005e-168,NaN,NaN


In [6]:
# Get most and least stable core genes based on label
pao1_most_stable_genes = list(
    pao1_similarity_scores[pao1_similarity_scores["label"] == "most stable"].index
)
pao1_least_stable_genes = list(
    pao1_similarity_scores[pao1_similarity_scores["label"] == "least stable"].index
)

In [7]:
# For each KEGG pathway, perform stat test, save p-values to get corrected p-values, report stats per pathway
def KEGG_enrichment_of_stable_genes(similarity_score_df, gene_list, kegg_df):
    """
    This function performs a KEGG enrichment using most or least stable genes,
    provided in `gene_list`
    """

    all_genes = set(similarity_score_df.index)
    module_genes = set(gene_list)
    not_module_genes = all_genes.difference(module_genes)

    rows = []
    # Find the KEGG pathway with significant over-representation
    for kegg_name in kegg_df.index:
        num_kegg_genes = kegg_df.loc[kegg_name, 1]
        kegg_genes = set(kegg_df.loc[kegg_name, 2])
        not_kegg_genes = all_genes.difference(kegg_genes)

        # Make contingency table
        # ---------------------| most stable  | not most stable
        # in KEGG pathway      | # genes      | # genes
        # not in KEGG pathway  | # genes     | # genes
        module_kegg_genes = module_genes.intersection(kegg_genes)
        not_module_kegg_genes = not_module_genes.intersection(kegg_genes)
        module_not_kegg_genes = module_genes.intersection(not_kegg_genes)
        not_module_not_kegg_genes = not_module_genes.intersection(not_kegg_genes)

        observed_contingency_table = np.array(
            [
                [len(module_kegg_genes), len(not_module_kegg_genes)],
                [len(module_not_kegg_genes), len(not_module_not_kegg_genes)],
            ]
        )
        # Fisher's exact test
        oddsr, pval = scipy.stats.fisher_exact(
            observed_contingency_table, alternative="greater"
        )
        # chi2 test will not accept 0 counts for the contingency table
        # chi2, pval, dof, expected_counts = scipy.stats.chi2_contingency(
        #    observed_contingency_table
        # )
        # print(oddsr, pval)

        rows.append(
            {
                "enriched KEGG pathway": kegg_name,
                "odds ratio": oddsr,
                "p-value": pval,
                "num shared genes": len(module_kegg_genes),
                "size gene set": len(module_genes),
                "size KEGG pathway": num_kegg_genes,
            }
        )

    enrichment_df = pd.DataFrame(rows)

    # Get corrected pvalues
    (
        reject_,
        pvals_corrected_,
        alphacSidak,
        alphacBonf,
    ) = statsmodels.stats.multitest.multipletests(
        enrichment_df["p-value"].values,
        alpha=0.05,
        method="fdr_bh",
        is_sorted=False,
    )

    enrichment_df["corrected p-value"] = pvals_corrected_

    return enrichment_df

In [8]:
pao1_most_stable_enrichment = KEGG_enrichment_of_stable_genes(
    pao1_similarity_scores, pao1_most_stable_genes, pao1_pathways
)

In [9]:
pao1_least_stable_enrichment = KEGG_enrichment_of_stable_genes(
    pao1_similarity_scores, pao1_least_stable_genes, pao1_pathways
)

In [10]:
print(pao1_most_stable_enrichment.shape)
pao1_most_stable_enrichment.sort_values(by="corrected p-value").head()

(169, 7)


,enriched KEGG pathway,odds ratio,p-value,num shared genes,size gene set,size KEGG pathway,corrected p-value
168,"KEGG-Module-M00178: Ribosome, bacteria",24.968492,3.834788e-28,38,473,56,3.240396e-26
50,KEGG-Pathway-pae03010: Ribosome,24.968492,3.834788e-28,38,473,68,3.240396e-26
32,KEGG-Pathway-pae03070: Bacterial secretion system,4.859193,2.662838e-09,27,473,92,1.500066e-07
114,KEGG-Module-M00332: Type III secretion system,16.561224,4.148334e-08,11,473,18,1.752671e-06
112,KEGG-Module-M00334: Type VI secretion system,6.793523,1.398506e-07,16,473,45,4.726952e-06


In [11]:
print(pao1_least_stable_enrichment.shape)
pao1_least_stable_enrichment.sort_values(by="corrected p-value").head()

(169, 7)


,enriched KEGG pathway,odds ratio,p-value,num shared genes,size gene set,size KEGG pathway,corrected p-value
0,KEGG-Pathway-pae00072: Synthesis and degradati...,0.000,1.000000,0,201,10,1.0
107,KEGG-Module-M00299: Spermidine/putrescine tran...,0.000,1.000000,0,201,12,1.0
108,KEGG-Module-M00222: Phosphate transport system,0.000,1.000000,0,201,5,1.0
109,KEGG-Module-M00053: Pyrimidine deoxyribonuleot...,0.000,1.000000,0,201,9,1.0
110,KEGG-Module-M00050: Guanine ribonucleotide bio...,5.143,0.205401,1,201,6,1.0


In [12]:
# Save
pao1_most_stable_enrichment.to_csv("pao1_most_stable_enrichment.tsv", sep="\t")
pao1_least_stable_enrichment.to_csv("pao1_least_stable_enrichment.tsv", sep="\t")

**Takeaway:**
* There does not appear to be any enriched KEGG pathways in the least stable genes.
    * What does this mean about the role of these least stable core genes? Maybe they are spread across multiple pathways?
    * Based on the dataframe created in the [previous notebook](2_find_KEGG_associations.ipynb) like many least stable core genes are not found in any KEGG pathway, but there are some that are found in many KEGG pathways: https://docs.google.com/spreadsheets/d/1SqEyBvutfbsOTo4afg9GiEzP32ZKplkN1a6MpAQBvZI/edit#gid=1943176121
* The most stable core genes are significantly enriched KEGG pathways include Ribosome (commonly enriched in humans), secretion system, metabolism/Krebs cycle
    * These KEGG pathways represent some of the essential functions for Pa, so it makes sense that they are enriched amongst the set of stable core genes whose transcriptional relationships don’t vary across strains.
    * Only some metabolisms and not others, is that interesting?